In [24]:
import pandas as pd
import joblib as jb

In [25]:
model = jb.load("models/10-xgb_more_examples.pkl")

items = pd.read_csv("data/items.csv")

In [26]:
df = pd.read_csv("data/test.csv")
df.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [27]:
df = pd.merge(df, items, on="item_id", how="inner").drop(columns=["item_name"])
df.sample(10)

,ID,shop_id,item_id,item_category_id
85272,63230,22,1238,58
84443,119310,53,2639,55
64059,47425,31,14521,55
199057,101639,19,1016,67
159582,126199,52,20881,72
118102,206811,39,20599,72
32738,102779,42,13272,47
146585,28990,7,6870,58
167154,187579,34,7884,6
60411,77938,15,11769,40


In [28]:
features_folder = "generated/features_prediction/"

features_dict = {
    features_folder + "features_shop_and_categories.csv":["shop_id", "item_category_id"],
    features_folder + "features_categories.csv":"item_category_id",
    features_folder + "features_shops.csv":"shop_id",
    features_folder + "features_items_and_shop.csv": ["item_id", "shop_id"],
    features_folder + "features_items.csv":"item_id"    
}

In [29]:
print(len(df))
for features_file in features_dict:
    df_features = pd.read_csv(features_file)
    joining_cols = features_dict[features_file]
    df = pd.merge(df, df_features, on=joining_cols, how="left")
print(len(df))
df.sample(10)

214200
214200


,ID,shop_id,item_id,item_category_id,hist_sales_sum_cat_by_shop,hist_sales_mean_cat_by_shop,hist_sales_std_cat_by_shop,hist_sales_min_cat_by_shop,hist_sales_max_cat_by_shop,sold_cat_last_month_by_shop,...,purchases_item_in_shop_month_6,month_last_purchase_of_item_in_shop,month_first_purchase_of_item_in_shop,total_months_with_sells_by_item_and_shop,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell
210603,81514,15,12065,38,1356.0,39.882353,15.286684,8.0,75.0,25.0,...,NaN,NaN,NaN,NaN,599.0,438.512500,199.90,181.605839,399.10,25.0
102117,78931,15,10315,40,11461.0,337.088235,164.635201,98.0,623.0,144.0,...,NaN,32.0,32.0,1.0,149.0,149.000000,149.00,0.000000,0.00,15.0
203555,122146,53,17981,37,2161.0,63.558824,19.118754,33.0,111.0,49.0,...,NaN,NaN,NaN,NaN,399.0,390.347826,260.00,33.199208,139.00,25.0
90880,175563,35,13667,69,532.0,15.647059,12.593607,6.0,75.0,21.0,...,NaN,NaN,NaN,NaN,999.0,999.000000,999.00,0.000000,0.00,4.0
60898,205449,39,13238,47,210.0,42.000000,40.792156,1.0,96.0,96.0,...,NaN,33.0,32.0,2.0,799.0,789.050833,561.00,40.975049,238.00,20.0
122579,120218,53,20189,49,149.0,4.656250,2.869297,1.0,12.0,2.0,...,NaN,NaN,NaN,NaN,420.0,416.553571,227.00,25.790710,193.00,12.0
26050,51620,26,1570,23,2697.0,79.323529,36.680207,32.0,162.0,34.0,...,6.0,31.0,22.0,7.0,2999.0,2485.744132,999.00,702.276514,2000.00,48.0
210539,183512,38,12569,37,2088.0,61.411765,21.828696,27.0,119.0,38.0,...,NaN,15.0,11.0,2.0,499.0,444.183388,186.97,72.136518,312.03,36.0
173335,9227,4,19331,49,57.0,2.192308,1.744001,1.0,9.0,NaN,...,NaN,NaN,NaN,NaN,350.0,349.453125,280.00,5.636260,70.00,23.0
14621,25848,7,14127,38,986.0,29.000000,11.651505,11.0,58.0,16.0,...,NaN,24.0,21.0,2.0,999.0,930.934363,435.00,130.665341,564.00,46.0


In [30]:
month_to_predict = 34

df["hist_sales_sum_cat_by_shop"] = df["hist_sales_sum_cat_by_shop"].fillna(0)
df["hist_sales_mean_cat_by_shop"] = df["hist_sales_mean_cat_by_shop"].fillna(0)
df["hist_sales_max_cat_by_shop"] = df["hist_sales_max_cat_by_shop"].fillna(0)
df["hist_sales_min_cat_by_shop"] = df["hist_sales_min_cat_by_shop"].fillna(0)
df["sold_cat_last_month_by_shop"] = df["sold_cat_last_month_by_shop"].fillna(0)
df["tot_cat_by_shops"] = df["tot_cat_by_shops"].fillna(df["tot_cat_by_shops"].mean())
df["items_sold_last_month"] = df["items_sold_last_month"].fillna(0)
df["hist_sales_mean_item_by_shop"] = df["hist_sales_mean_item_by_shop"].fillna(0)
df["hist_sales_sum_item_by_shop"] = df["hist_sales_sum_item_by_shop"].fillna(0)
df["hist_sales_max_item_by_shop"] = df["hist_sales_max_item_by_shop"].fillna(0)
df["hist_sales_min_item_by_shop"] = df["hist_sales_min_item_by_shop"].fillna(0)
df["purchases_item_in_shop_last_month"] = df["purchases_item_in_shop_last_month"].fillna(0)
for x in range(5):
    df["purchases_item_in_shop_month_" + str(x+2)].fillna(0, inplace=True)
df.sample(10)

,ID,shop_id,item_id,item_category_id,hist_sales_sum_cat_by_shop,hist_sales_mean_cat_by_shop,hist_sales_std_cat_by_shop,hist_sales_min_cat_by_shop,hist_sales_max_cat_by_shop,sold_cat_last_month_by_shop,...,purchases_item_in_shop_month_6,month_last_purchase_of_item_in_shop,month_first_purchase_of_item_in_shop,total_months_with_sells_by_item_and_shop,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell
93339,78722,15,19045,55,6482.0,190.647059,55.189416,96.0,340.0,121.0,...,1.0,29.0,5.0,8.0,299.0,251.679803,173.00,20.294635,126.00,43.0
201649,40501,12,14406,37,2860.0,84.117647,46.381661,18.0,219.0,155.0,...,0.0,NaN,NaN,NaN,249.0,199.561290,141.00,19.249725,108.00,29.0
28819,36386,12,13463,19,2992.0,88.000000,48.885767,32.0,201.0,39.0,...,0.0,31.0,31.0,1.0,1699.0,1692.062464,1220.31,57.627504,478.69,30.0
213362,15280,6,19889,40,13643.0,401.264706,167.170990,166.0,758.0,216.0,...,0.0,6.0,4.0,2.0,149.0,146.566667,76.00,13.177313,73.00,31.0
57102,123759,52,19290,67,688.0,20.235294,14.244741,4.0,80.0,7.0,...,0.0,32.0,27.0,3.0,999.0,994.901691,645.59,34.707057,353.41,38.0
170142,4051,5,4579,55,4892.0,148.242424,35.075838,84.0,213.0,106.0,...,0.0,33.0,4.0,4.0,449.0,315.079872,225.00,49.260656,224.00,47.0
53021,87962,18,6836,55,6004.0,176.588235,66.123905,100.0,325.0,182.0,...,0.0,NaN,NaN,NaN,299.0,292.216800,177.84,27.158325,121.16,20.0
63911,149421,59,13635,69,345.0,10.454545,9.294121,1.0,38.0,13.0,...,0.0,NaN,NaN,NaN,999.0,999.000000,999.00,0.000000,0.00,6.0
118766,166027,36,20439,72,13.0,13.000000,NaN,13.0,13.0,13.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36051,77358,15,15553,55,6482.0,190.647059,55.189416,96.0,340.0,121.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df.isna().sum()

ID                                               0
shop_id                                          0
item_id                                          0
item_category_id                                 0
hist_sales_sum_cat_by_shop                       0
hist_sales_mean_cat_by_shop                      0
hist_sales_std_cat_by_shop                   29050
hist_sales_min_cat_by_shop                       0
hist_sales_max_cat_by_shop                       0
sold_cat_last_month_by_shop                      0
total_shops_of_category                          0
total_items_of_category                          0
tot_cat_by_shops                                 0
items_sold_last_month                            0
hist_sales_mean_item_by_shop                     0
hist_sales_sum_item_by_shop                      0
hist_sales_std_item_by_shop                 129766
hist_sales_max_item_by_shop                      0
hist_sales_min_item_by_shop                      0
purchases_item_in_shop_last_mon

In [32]:
features = df

In [33]:
ids_cols = ["ID"]
preds = [x for x in features.columns if x not in ids_cols]
preds

predictors = ['shop_id',
 'item_id',
 'item_category_id',
 'hist_sales_sum_cat_by_shop',
 'hist_sales_mean_cat_by_shop',
 'hist_sales_std_cat_by_shop',
 'hist_sales_min_cat_by_shop',
 'hist_sales_max_cat_by_shop',
 'sold_cat_last_month_by_shop',
 'total_shops_of_category',
 'total_items_of_category',
 'tot_cat_by_shops',
 'items_sold_last_month',
 'hist_sales_mean_item_by_shop',
 'hist_sales_sum_item_by_shop',
 'hist_sales_std_item_by_shop',
 'hist_sales_max_item_by_shop',
 'hist_sales_min_item_by_shop',
 'purchases_item_in_shop_last_month',
 'purchases_item_in_shop_month_2',
 'purchases_item_in_shop_month_3',
 'purchases_item_in_shop_month_4',
 'purchases_item_in_shop_month_5',
 'purchases_item_in_shop_month_6',
 'month_last_purchase_of_item_in_shop',
 'month_first_purchase_of_item_in_shop',
 'total_months_with_sells_by_item_and_shop',
 'item_price_max',
 'item_price_mean',
 'item_price_min',
 'item_price_std',
 'diff_price_max_min',
 'total_shops_item_is_sell']


sorted(preds) == sorted(predictors)

True

## XGBOOST MODEL

In [34]:
X = df[predictors]
ids = df[ids_cols]

In [35]:
X["predictions"] = model.predict(X)
ids["item_cnt_month"] = X["predictions"]
ids.head()

/home/urielkelman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ID,item_cnt_month
0,0,1.184391
1,5100,1.494399
2,10200,1.226885
3,15300,1.394485
4,20400,1.079360


In [36]:
sub_1 = ids
sub_1.head()

,ID,item_cnt_month
0,0,1.184391
1,5100,1.494399
2,10200,1.226885
3,15300,1.394485
4,20400,1.079360


In [26]:
sub_1.to_csv("submissions/11-xgdeeper.csv", index=False)

## RANDOM FOREST MODEL

### Analyze NA values

In [10]:
df.isna().sum()

ID                                               0
shop_id                                          0
item_id                                          0
item_category_id                                 0
hist_sales_sum_cat_by_shop                       0
hist_sales_mean_cat_by_shop                      0
hist_sales_std_cat_by_shop                   29050
hist_sales_min_cat_by_shop                       0
hist_sales_max_cat_by_shop                       0
sold_cat_last_month_by_shop                      0
total_shops_of_category                          0
total_items_of_category                          0
tot_cat_by_shops                                 0
items_sold_last_month                            0
hist_sales_mean_item_by_shop                     0
hist_sales_sum_item_by_shop                      0
hist_sales_std_item_by_shop                 129766
hist_sales_max_item_by_shop                      0
hist_sales_min_item_by_shop                      0
purchases_item_in_shop_last_mon

In [11]:
# For item_price features, take main of category
grouped = df.groupby("item_category_id").agg({"item_price_max":"mean", "item_price_mean":"mean", "item_price_min":"mean", "item_price_std":"mean", "diff_price_max_min":"mean"}).reset_index()

df["id_help"] = [x for x in range(len(df))]

df_item_price_na = df[df.item_price_max.isna()]
df_item_price_na.drop(columns=["item_price_max", "item_price_mean", "item_price_std", "item_price_min", "diff_price_max_min"], inplace=True)
print(len(df_item_price_na))

15246


/home/urielkelman/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
df_item_price_na = pd.merge(df_item_price_na, grouped, on="item_category_id", how="inner")
print(len(df_item_price_na))

df_item_price_na = df_item_price_na[df.columns]
print(len(df_item_price_na))
df_item_price_na.columns == df.columns

15246
15246


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [13]:
print(len(df))
df = df_item_price_na.append(df).drop_duplicates(subset=["id_help"], keep="first").drop(columns=["id_help"])
print(len(df))
df.head()

214200
214200


,ID,shop_id,item_id,item_category_id,hist_sales_sum_cat_by_shop,hist_sales_mean_cat_by_shop,hist_sales_std_cat_by_shop,hist_sales_min_cat_by_shop,hist_sales_max_cat_by_shop,sold_cat_last_month_by_shop,...,purchases_item_in_shop_month_6,month_last_purchase_of_item_in_shop,month_first_purchase_of_item_in_shop,total_months_with_sells_by_item_and_shop,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell
0,1,5,5320,55,4892.0,148.242424,35.075838,84.0,213.0,106.0,...,0.0,NaN,NaN,NaN,334.015773,305.193119,192.565158,25.061594,141.450615,NaN
1,5101,4,5320,55,4952.0,145.647059,42.177750,28.0,229.0,106.0,...,0.0,NaN,NaN,NaN,334.015773,305.193119,192.565158,25.061594,141.450615,NaN
2,10201,6,5320,55,12104.0,356.000000,103.282197,157.0,623.0,221.0,...,0.0,NaN,NaN,NaN,334.015773,305.193119,192.565158,25.061594,141.450615,NaN
3,15301,3,5320,55,2885.0,84.852941,24.640607,37.0,126.0,38.0,...,0.0,NaN,NaN,NaN,334.015773,305.193119,192.565158,25.061594,141.450615,NaN
4,20401,2,5320,55,363.0,11.709677,22.767218,1.0,99.0,2.0,...,0.0,NaN,NaN,NaN,334.015773,305.193119,192.565158,25.061594,141.450615,NaN


In [14]:
df["hist_sales_std_cat_by_shop"] = df["hist_sales_std_cat_by_shop"].fillna(0)
df["hist_sales_std_item_by_shop"] = df["hist_sales_std_item_by_shop"].fillna(0)
df["month_last_purchase_of_item_in_shop"] = df["month_last_purchase_of_item_in_shop"].fillna(-1)
df["month_first_purchase_of_item_in_shop"] = df["month_first_purchase_of_item_in_shop"].fillna(-1)
df["item_price_std"] = df["item_price_std"].fillna(0)
df["total_shops_item_is_sell"] = df["total_shops_item_is_sell"].fillna(0)
df["total_shops_of_category"] = df["total_shops_of_category"].fillna(0)
df["total_items_of_category"] = df["total_items_of_category"].fillna(0)
df["total_months_with_sells_by_item_and_shop"] = df["total_months_with_sells_by_item_and_shop"].fillna(0)

In [15]:
df.isna().sum()

ID                                           0
shop_id                                      0
item_id                                      0
item_category_id                             0
hist_sales_sum_cat_by_shop                   0
hist_sales_mean_cat_by_shop                  0
hist_sales_std_cat_by_shop                   0
hist_sales_min_cat_by_shop                   0
hist_sales_max_cat_by_shop                   0
sold_cat_last_month_by_shop                  0
total_shops_of_category                      0
total_items_of_category                      0
tot_cat_by_shops                             0
items_sold_last_month                        0
hist_sales_mean_item_by_shop                 0
hist_sales_sum_item_by_shop                  0
hist_sales_std_item_by_shop                  0
hist_sales_max_item_by_shop                  0
hist_sales_min_item_by_shop                  0
purchases_item_in_shop_last_month            0
purchases_item_in_shop_month_2               0
purchases_ite

In [16]:
df["item_price_max"] = df["item_price_max"].fillna(df["item_price_max"].mean()) 
df["item_price_mean"] = df["item_price_mean"].fillna(df["item_price_mean"].mean()) 
df["item_price_min"] = df["item_price_min"].fillna(df["item_price_min"].mean()) 
df["diff_price_max_min"] = df["diff_price_max_min"].fillna(df["diff_price_max_min"].mean()) 
df.isna().sum()

ID                                          0
shop_id                                     0
item_id                                     0
item_category_id                            0
hist_sales_sum_cat_by_shop                  0
hist_sales_mean_cat_by_shop                 0
hist_sales_std_cat_by_shop                  0
hist_sales_min_cat_by_shop                  0
hist_sales_max_cat_by_shop                  0
sold_cat_last_month_by_shop                 0
total_shops_of_category                     0
total_items_of_category                     0
tot_cat_by_shops                            0
items_sold_last_month                       0
hist_sales_mean_item_by_shop                0
hist_sales_sum_item_by_shop                 0
hist_sales_std_item_by_shop                 0
hist_sales_max_item_by_shop                 0
hist_sales_min_item_by_shop                 0
purchases_item_in_shop_last_month           0
purchases_item_in_shop_month_2              0
purchases_item_in_shop_month_3    

In [17]:
features = df

In [18]:
ids_cols = ["ID"]
preds = [x for x in features.columns if x not in ids_cols]
preds

predictors = ['shop_id',
 'item_id',
 'item_category_id',
 'hist_sales_sum_cat_by_shop',
 'hist_sales_mean_cat_by_shop',
 'hist_sales_std_cat_by_shop',
 'hist_sales_min_cat_by_shop',
 'hist_sales_max_cat_by_shop',
 'sold_cat_last_month_by_shop',
 'total_shops_of_category',
 'total_items_of_category',
 'tot_cat_by_shops',
 'items_sold_last_month',
 'hist_sales_mean_item_by_shop',
 'hist_sales_sum_item_by_shop',
 'hist_sales_std_item_by_shop',
 'hist_sales_max_item_by_shop',
 'hist_sales_min_item_by_shop',
 'purchases_item_in_shop_last_month',
 'purchases_item_in_shop_month_2',
 'purchases_item_in_shop_month_3',
 'purchases_item_in_shop_month_4',
 'purchases_item_in_shop_month_5',
 'purchases_item_in_shop_month_6',
 'month_last_purchase_of_item_in_shop',
 'month_first_purchase_of_item_in_shop',
 'total_months_with_sells_by_item_and_shop',
 'item_price_max',
 'item_price_mean',
 'item_price_min',
 'item_price_std',
 'diff_price_max_min',
 'total_shops_item_is_sell']


sorted(preds) == sorted(predictors)

True

In [23]:
X = df[predictors]
ids = df[ids_cols]

X["predictions"] = model.predict(X)
ids["item_cnt_month"] = X["predictions"]

sub_2 = ids.sort_values(by="ID")
sub_2.head()

/home/urielkelman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,ID,item_cnt_month
0,0,1.602860
0,1,1.321108
84,2,2.148142
126,3,1.281490
1596,4,3.786940


In [22]:
sub_2.to_csv("submissions/12-rf.csv", index=False)

## MERGE XGBOOST AND RF

In [37]:
sub_1.head()

,ID,item_cnt_month
0,0,1.184391
1,5100,1.494399
2,10200,1.226885
3,15300,1.394485
4,20400,1.079360


In [38]:
sub_2.head()

,ID,item_cnt_month
0,0,1.602860
0,1,1.321108
84,2,2.148142
126,3,1.281490
1596,4,3.786940


In [39]:
sub = pd.merge(sub_1, sub_2, on="ID", how="inner")
sub.head()

,ID,item_cnt_month_x,item_cnt_month_y
0,0,1.184391,1.602860
1,5100,1.494399,1.525391
2,10200,1.226885,1.656878
3,15300,1.394485,1.579156
4,20400,1.079360,1.675472


In [40]:
sub["item_cnt_month"] = sub["item_cnt_month_x"] * 0.5 + sub["item_cnt_month_y"] * 0.5
sub.drop(columns=["item_cnt_month_x", "item_cnt_month_y"], inplace=True)
sub.head()

,ID,item_cnt_month
0,0,1.393625
1,5100,1.509895
2,10200,1.441881
3,15300,1.486820
4,20400,1.377416


In [41]:
sub.sort_values(by="ID", inplace=True)
sub.head()

,ID,item_cnt_month
0,0,1.393625
42,1,1.350345
84,2,1.870072
126,3,1.267259
168,4,4.038741


In [42]:
sub.to_csv("submissions/13 - avg.csv", index=False)

In [43]:
from math import modf

In [56]:
def downgrade_or_upgrade_if_necessary(x):
    m = modf(x)
    if m[0] < 0.2:
        return m[1]
    if m[0] > 0.8:
        return m[1] + 1
    return x

In [57]:
sub["item_cnt_month"] = sub["item_cnt_month"].map(lambda x: downgrade_or_upgrade_if_necessary(x))
sub.sample(15)

,ID,item_cnt_month
198308,137321,1.000000
156201,19019,1.634725
208591,101866,1.000000
120699,171173,1.000000
66977,149494,1.000000
134147,212293,1.000000
173271,111225,1.000000
206260,208910,1.000000
5259,46025,2.000000
124545,79465,1.000000


In [ ]:
sub.to_csv("submissions/14 - rounding.csv", index=F)